In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
data = pd.read_excel('data lama 4 hari.xlsx', sheet_name=None)

In [ ]:
# BAGIAN 1: LOAD DAN CLEANING DATA BARU (NOVEMBER 6-9)

def parse_datetime_safe(col):
    """
    Fungsi untuk parsing datetime dengan format DD/MM/YYYY HH:MM:SS
    
    Parameters:
    - col: pandas Series berisi string tanggal
    
    Returns:
    - pandas Series berisi datetime yang sudah di-parse
    """
    # Konversi ke string dan hapus spasi
    col = col.astype(str).str.strip()
    
    # Parse dengan format eksplisit DD/MM/YYYY
    parsed = pd.to_datetime(col, format='%d/%m/%Y %H:%M:%S', errors='coerce', dayfirst=True)
    
    # Jika lebih dari 50% gagal, coba auto-detect format
    if parsed.isna().sum() > len(col) * 0.5:
        parsed = pd.to_datetime(col, errors='coerce', dayfirst=True, infer_datetime_format=True)
    
    return parsed


def load_data_baru(data):
    """
    Load data BARU dari Excel sheet tertentu
    Hanya untuk data November 6-9, 2025
    
    Parameters:
    - filepath: path ke file Excel
    - sheet_name: nama sheet (default 'data lama 4 hari')
    
    Returns:
    - df: DataFrame lengkap dengan semua data
    - df_active: DataFrame hanya dengan ACC ON
    """
    print("="*60)
    print(f"LOADING DATA: {data}")
    print("="*60)
    
    # Load data dari Excel
    df = data['data lama 4 hari']
    
    print(f"\n[1/5] Data loaded: {len(df)} rows")
    
    # Parse kolom GPSTIME
    print("[2/5] Parsing GPSTIME...")
    df['GPSTIME'] = parse_datetime_safe(df['GPSTIME'])
    
    # Parse kolom RECEIVE TIME jika ada
    if 'RECEIVE TIME' in df.columns:
        df['RECEIVE TIME'] = parse_datetime_safe(df['RECEIVE TIME'])
    
    # Konversi kolom numerik
    print("[3/5] Converting numeric columns...")
    df['VALUE FUEL SENSOR'] = pd.to_numeric(df['VALUE FUEL SENSOR'], errors='coerce')
    df['VOLTAGE FUEL SENSOR'] = pd.to_numeric(df['VOLTAGE FUEL SENSOR'], errors='coerce')
    df['ODOMETER'] = pd.to_numeric(df['ODOMETER'], errors='coerce')
    df['SPEED'] = pd.to_numeric(df['SPEED'], errors='coerce')
    
    # Hapus baris dengan GPSTIME invalid
    print("[4/5] Removing invalid rows...")
    initial_count = len(df)
    df = df.dropna(subset=['GPSTIME'])
    removed = initial_count - len(df)
    if removed > 0:
        print(f"    Removed {removed} rows with invalid GPSTIME")
    
    # Sort berdasarkan waktu
    print("[5/5] Sorting and filtering...")
    df = df.sort_values('GPSTIME').reset_index(drop=True)
    
    # Filter hanya ACC ON
    df_active = df[df['ACC'] == 'ON'].copy()
    
    # Tampilkan ringkasan
    print("\n" + "-"*60)
    print(f"Total data: {len(df):,}")
    print(f"Data ACC ON: {len(df_active):,}")
    print(f"Rentang waktu: {df_active['GPSTIME'].min()} s/d {df_active['GPSTIME'].max()}")
    
    # Validasi bulan dan hari
    months = df_active['GPSTIME'].dt.month.unique()
    days = sorted(df_active['GPSTIME'].dt.day.unique())
    print(f"Bulan: {months} (November = 11)")
    print(f"Hari: {days}")
    print("="*60)
    
    return df, df_active


# LOAD DATA BARU
filepath = 'Data W9371UM ULTRASONIK.xlsx'
df_baru, df_baru_active = load_data_baru(data)